# Workflows in Python
Description : This notebook was made to following along with the articles written by Caitlin Malone on ODSC. Through these articles, Caitlin works with data collected for the "Pump it Up: Data Mining the Water Table" drivendata competition [[Link](https://www.drivendata.org/competitions/7/)] .

[Link to Article (Part 1)](https://www.opendatascience.com/blog/preprocessing-data-a-python-workflow-part-1/)

### Part 1 Notes

- read in data
- transformed features and labels to make the data amenable to machine learning
- picked a modeling strategy (classification)
- made a train/test split (this was done implicitly when I called cross_val_score)
- evaluated several models for identifying wells that are failed or in danger of failing

## Workflows in Python Part 1: Preprocessing Data

In [22]:
# Imports
import pandas as pd
import numpy as np

import wget

from sklearn import preprocessing
import sklearn.linear_model
import sklearn.cross_validation
import sklearn.tree
import sklearn.ensemble

In [2]:
# Saving the Data (Just in case links break)
# wget.download('https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv', out='train_features.csv')
# wget.download('https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv', out='train_labels.csv')
# wget.download('https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv', out='test_values.csv')

In [3]:
# Getting the data
features_df = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv')
labels_df = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')
test_set_values = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')

In [4]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
id              59400 non-null int64
status_group    59400 non-null object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [5]:
print( labels_df.head(10) )

      id    status_group
0  69572      functional
1   8776      functional
2  34310      functional
3  67743  non functional
4  19728      functional
5   9944      functional
6  19816  non functional
7  54551  non functional
8  53934  non functional
9  46144      functional


In [6]:
#Mapping
map_dict = {"functional": 2,
            "functional needs repair": 1,
            "non functional": 0
            }

In [7]:
labels_df['status_group'] = labels_df['status_group'].map(map_dict)
print( labels_df.head() )

      id  status_group
0  69572             2
1   8776             2
2  34310             2
3  67743             0
4  19728             2


In [8]:
# Finding out which columns are object type and oneHotEncoder in sklearn or get_dummies() in pandas.

list_of_columns = list(features_df.columns)
cols_to_transform = []

for col in list_of_columns:
    if features_df[col].dtype == 'object':
        cols_to_transform.append(col)

In [9]:
def transform_feature( df, column_name ):
    unique_values = set( df[column_name].tolist() )
    transformer_dict = {}
    for ii, value in enumerate(unique_values):
        transformer_dict[value] = ii
    df[column_name] = df[column_name].map( transformer_dict )
    return df

In [10]:
for column in cols_to_transform:
    features_df = transform_feature( features_df, column )

In [11]:
print( features_df.head() )

      id  amount_tsh  date_recorded  funder  gps_height  installer  longitude  \
0  69572      6000.0             10      96        1390        115  34.938093   
1   8776         0.0             69     318        1399        124  34.698766   
2  34310        25.0            255    1702         686       1176  37.460664   
3  67743         0.0            319    1336         263        258  38.486161   
4  19728         0.0            194     189           0       1482  31.130847   

    latitude  wpt_name  num_private          ...            payment_type  \
0  -9.856322     12813            0          ...                       3   
1  -2.147466     22016            0          ...                       6   
2  -3.821329     23714            0          ...                       2   
3 -11.155298      1242            0          ...                       6   
4  -1.825359      7360            0          ...                       6   

   water_quality  quality_group  quantity  quantity_grou

In [12]:
features_df.drop("date_recorded", axis=1, inplace=True)

In [13]:
print(features_df.columns.values)

['id' 'amount_tsh' 'funder' 'gps_height' 'installer' 'longitude' 'latitude'
 'wpt_name' 'num_private' 'basin' 'subvillage' 'region' 'region_code'
 'district_code' 'lga' 'ward' 'population' 'public_meeting' 'recorded_by'
 'scheme_management' 'scheme_name' 'permit' 'construction_year'
 'extraction_type' 'extraction_type_group' 'extraction_type_class'
 'management' 'management_group' 'payment' 'payment_type' 'water_quality'
 'quality_group' 'quantity' 'quantity_group' 'source' 'source_type'
 'source_class' 'waterpoint_type' 'waterpoint_type_group']


In [19]:
X = features_df.as_matrix()
y = labels_df["status_group"].tolist()

This splits my data into three equal portions, trains on two of them, and tests on the third. This process repeats three times. That’s why three numbers get printed in the code block below.

In [21]:
clf = sklearn.linear_model.LogisticRegression()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

[ 0.63560606  0.64883838  0.645     ]


In [23]:
## DecisionTree and RandomForest

clf = sklearn.tree.DecisionTreeClassifier()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

clf = sklearn.ensemble.RandomForestClassifier()
score = sklearn.cross_validation.cross_val_score( clf, X, y )
print( score )

[ 0.72792929  0.73222222  0.72580808]
[ 0.78489899  0.78429293  0.78323232]
